<a href="https://colab.research.google.com/github/L1KASA/big-data/blob/main/lab3/6409_%D0%A2%D0%B0%D1%85%D0%BC%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D0%90%D0%A8_%D0%9B%D0%A03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка среды

In [1]:
# Установка Java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Загрузка бинарного файла Apache Spark: Эта ссылка может измениться в зависимости от версии. Обновите эту ссылку перед использованием на последнюю версию
!wget -q https://archive.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

# Распаковка файла
!tar xf spark-3.0.2-bin-hadoop2.7.tgz

# Установка findspark: Добавляет PySpark в sys.path во время выполнения
!pip install -q findspark

# Установка pyspark
!pip install pyspark

# Добавление переменных среды
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# findspark определит расположение Spark в системе
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,107 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [61.2 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/ma

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [6]:
#!pip install findspark
import findspark
findspark.init()

In [4]:
!pip install pyspark py4j

# Импортирование нужных библиотек

In [5]:
import re
from typing import List
import pyspark.sql as sql
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, col, max, sum, desc, countDistinct
import os

# Инициализация сессии

In [10]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

In [11]:
spark = SparkSession.builder.appName("L2_reports_with_apache_spark").getOrCreate()

In [12]:
spark.version

'3.0.2'

# Загрузка данных

In [30]:
posts_path = '/content/posts_sample.xml'
programming_languages_path = '/content/programming-languages.csv'

In [31]:
posts_data = spark.read.format('xml').options(rowTag='row').load(posts_path)
print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [32]:
programming_languages_data = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(programming_languages_path)

# Работа с данными

## Прописываем функцию для определения языка программирования, связанного с каждым постом на основе тегов

In [33]:
def check_lang(x, csv_list):
    tags_lower = x._Tags.lower()
    language = next((language for language in csv_list if f"<{language.lower()}>" in tags_lower), None)
    return (x._Id, language) if language else None

## Извлечение названий языков программирования из CSV

In [34]:
csv_list = programming_languages_data.selectExpr("name").rdd.flatMap(lambda x: x).collect()

## Получение минимального и максимального года

In [35]:
min_year = int(posts_data.selectExpr("min(substring(_CreationDate, 0, 4))").first()[0])
max_year = int(posts_data.selectExpr("max(substring(_CreationDate, 0, 4))").first()[0])

print(min_year)
print(max_year)

2008
2019


## Анализ топ языков программирования по годам

In [36]:
result = {}

for year in range(int(min_year), int(max_year) + 1):
    # Фильтрация данных по году
    filtered_data = posts_data.filter((col("_Tags").isNotNull()) & (col("_CreationDate").substr(0, 4) == str(year)))

    # Применение функции check_lang к данным и фильтрация None значений
    mapped_data = filtered_data.rdd.map(lambda x: check_lang(x, csv_list)).filter(lambda x: x is not None)

    # Агрегация данных
    aggregated_data = mapped_data \
        .map(lambda x: (x[1], 1)) \
        .reduceByKey(lambda x, y: x + y) \
        .map(lambda x: (x[0], int(x[1])))  # Преобразование счетчика в целочисленный тип

    # Преобразование RDD в DataFrame
    sorted_data = spark.createDataFrame(aggregated_data, ["Language", f"{year}_year"]) \
        .orderBy(col(f"{year}_year").desc()) \
        .limit(10)

    result[year] = sorted_data
    print(f"Топ 10 языков программирования за {year} год:")
    result[year].show()

    # Сохранение результата в формате Parquet
    sorted_data.write.parquet(f"top_programming_languages_{year}.parquet")

Топ 10 языков программирования за 2008 год:
+----------+---------+
|  Language|2008_year|
+----------+---------+
|      Java|        4|
|      Ruby|        3|
|JavaScript|        2|
|         C|        2|
|    Groovy|        1|
|    Python|        1|
|       X++|        1|
|        Io|        1|
|       PHP|        1|
+----------+---------+

Топ 10 языков программирования за 2009 год:
+-----------+---------+
|   Language|2009_year|
+-----------+---------+
|       Java|       28|
|     Python|       22|
|        PHP|       22|
| JavaScript|       12|
|     Delphi|        7|
|       Ruby|        7|
|          C|        6|
|Objective-C|        5|
|    Haskell|        4|
|       Bash|        3|
+-----------+---------+

Топ 10 языков программирования за 2010 год:
+-----------+---------+
|   Language|2010_year|
+-----------+---------+
|       Java|       52|
| JavaScript|       44|
|        PHP|       42|
|     Python|       25|
|Objective-C|       23|
|          C|       20|
|       Ruby|  

## Остановка SparkSession

In [37]:
spark.stop()

## Проверка работы с файлами Parquet

In [38]:
import pandas as pd

# Чтение файла Parquet
df = pd.read_parquet("top_programming_languages_2009.parquet")

# Просмотр данных
print(df)


      Language  2009_year
0         Java         28
1       Python         22
2          PHP         22
3   JavaScript         12
4       Delphi          7
5         Ruby          7
6            C          6
7  Objective-C          5
8      Haskell          4
9         Bash          3
